In [ ]:
from SymbolicDSGE import ModelConfig, ModelParser, DSGESolver, FRED
from SymbolicDSGE.math_utils import HP_two_sided
import sympy as sp
from warnings import catch_warnings, simplefilter
import numpy as np
import pandas as pd

In [ ]:
conf = ModelParser("MODELS/POST82.yaml").get()

In [ ]:
with catch_warnings():
    simplefilter(action="ignore")
    mat = sp.Matrix(conf.equations.model)
mat

In [ ]:
sol = DSGESolver(conf)
comp = sol.compile(variable_order=conf.variables, n_state=3, n_exog=2)
conf.variables

In [ ]:
solved = sol.solve(
    comp,
    steady_state=np.asarray([0.0, 0.0, 0.0, 0.0, 0.0], dtype=float),
    log_linear=False,
)

In [ ]:
solved.policy.eig

In [ ]:
params = {
    p.name: float(conf.calibration.parameters[p])
    for p in conf.parameters
    if p in conf.calibration.parameters
}

# state at time t
s = np.array([0.05, 0.077, 0.06])  # or any test state
P = solved.policy.p
F = solved.policy.f

# controls at time t (jump variables)
c = F @ s

cur = np.concatenate([s, c])

# expected next state (NO shock)
s1 = P @ s
c1 = F @ s1
fwd = np.concatenate([s1, c1])

res = solved.compiled.equations(fwd, cur, params)
print(res)

In [ ]:
solved.transition_plot(25, ["g", "z"], 1.0, observables=True)

In [12]:
sim_shocks = np.array([[1.0, 1.0], *np.zeros((24, 2))])
solved.sim(25, sim_shocks, observables=True)

{'g': array([0.        , 1.        , 0.83000003, 0.68890005, 0.57178706,
        0.47458328, 0.39390414, 0.32694045, 0.27136058, 0.22522929,
        0.18694032, 0.15516047, 0.12878319, 0.10689005, 0.08871875,
        0.07363656, 0.06111835, 0.05072823, 0.04210443, 0.03494668,
        0.02900575, 0.02407477, 0.01998206, 0.01658511, 0.01376564,
        0.01142548]),
 'z': array([0.        , 1.        , 0.84999992, 0.72249987, 0.61412483,
        0.52200606, 0.44370511, 0.37714931, 0.32057688, 0.27249032,
        0.23161675, 0.19687422, 0.16734307, 0.1422416 , 0.12090535,
        0.10276954, 0.0873541 , 0.07425098, 0.06311332, 0.05364632,
        0.04559937, 0.03875946, 0.03294554, 0.0280037 , 0.02380315,
        0.02023267]),
 'r': array([0.        , 0.        , 0.48060113, 0.53692545, 0.48745108,
        0.41903176, 0.35418962, 0.29776799, 0.24991054, 0.20964626,
        0.17585806, 0.14752611, 0.12377322, 0.10385856, 0.08716015,
        0.07315678, 0.06141189, 0.05155986, 0.04329448, 0

In [ ]:
# fred test
f = FRED(key_name="FRED_KEY")
df = f.get_frame(
    series_ids=["GDPC1", "CPIAUCSL", "FEDFUNDS"],
    date_range=("1960-01-01", "1997-10-01"),
)

In [ ]:
time_idx = pd.date_range(start="1960-01-01", end="1997-10-01", freq="QS")
df = df.reindex(time_idx)
df

In [ ]:
# Convert to model units
df["GDPC1"] = 100 * (np.log(df["GDPC1"]) - HP_two_sided(np.log(df["GDPC1"]), 1600)[0])
df["CPIAUCSL"] = 400 * np.log((df["CPIAUCSL"] / df["CPIAUCSL"].shift(1)).dropna())

In [ ]:
df = df.loc[df.index >= "1982-01-01"]

In [ ]:
x0 = [0.0, 0.0, df["FEDFUNDS"].iloc[0], df["CPIAUCSL"].iloc[0], df["GDPC1"].iloc[0]]
solved.sim(T=df.shape[0], shocks=None, observables=True, x0=x0)